Example on how to compute representation of Pharmacophores with GVP-GNN

In [2]:
import sys

sys.path.append('../')

In [3]:
from src.tacogfn.models import gvp_model
from utils import get_example_pharmacophore_datalist

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_list = get_example_pharmacophore_datalist()

In [5]:
# model = GVP_embedding((6, 3), (32, 0), (32, 1), (32, 0), seq_in=True).to(device)

embedding_channels = 128

model = gvp_model.GVP_embedding((24, 1), (embedding_channels, 16), (24, 1), (32, 1), seq_in=True)

In [12]:
data = data_list[0]
out = model((data.node_s, data.node_v), data.edge_index, (data.edge_s, data.edge_v), data.seq)